In [7]:
#installing important dependencies
# ultralytics for yolo , cv2 for capture setting and torch for handling tensors if needed
!pip install ultralytics
!pip install cv2 
!pip install torch 
!pip install matplotlib 
!pip install pycocotools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.5 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [8]:
import os
import glob
from pathlib import Path
from ultralytics import YOLO as yl
from collections import Counter
import random
from pycocotools.coco import COCO
import time
import json
import shutil
import cv2
import json
from PIL import Image
from io import BytesIO
import requests

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
def safe_download(url, save_path):
    save_path = Path(save_path)
    save_path.parent.mkdir(parents=True, exist_ok=True)
    if save_path.exists():
        return
    try:
        r = requests.get(url, stream=True, timeout=20)
        r.raise_for_status()
        with open(save_path, "wb") as f:
            for chunk in r.iter_content(1024*64):
                if chunk:
                    f.write(chunk)
    except Exception as e:
        print(f"[WARN] Could not fetch {url} -> {e}")


def map_category_to_3(name, supercat):
    n = (name or "").lower()
    s = (supercat or "").lower()

    # Plastic
    if any(x in n for x in ["plastic", "styrofoam", "polystyrene", "pet", "tetra pak", "wrapper", "bag", "straw"]):
        return 0
    if "plastic" in s:
        return 0

    # Metal
    if any(x in n for x in ["metal", "aluminium", "aluminum", "steel", "can", "foil", "tin"]):
        return 1
    if "metal" in s:
        return 1

    # Paper
    if any(x in n for x in ["paper", "cardboard", "carton", "napkin", "tissue", "cup", "notebook"]):
        return 2
    if "paper" in s:
        return 2

    return None


def prepare_taco_3class(root="dataset", n_images=120, seed=42):
    root = Path(root)
    root.mkdir(parents=True, exist_ok=True)

    # 1. Download annotations
    ann_url = "https://github.com/pedropro/TACO/raw/master/data/annotations.json"
    ann_file = root / "annotations.json"
    safe_download(ann_url, ann_file)

    # 2. Load COCO annotations
    coco = COCO(str(ann_file))
    img_ids = coco.getImgIds()
    random.seed(seed)
    random.shuffle(img_ids)
    selected = img_ids[:n_images]

    # 3. Create folders
    for split in ["train", "val"]:
        (root / "images" / split).mkdir(parents=True, exist_ok=True)
        (root / "labels" / split).mkdir(parents=True, exist_ok=True)

    # 4. Process images + labels
    kept_labels = 0
    for i, img_id in enumerate(selected):
        img_info = coco.loadImgs(img_id)[0]
        url = img_info.get("flickr_url") or img_info.get("coco_url")
        split = "train" if i < int(0.8 * n_images) else "val"
        fname = f"{img_id}.jpg"
        img_path = root / "images" / split / fname
        label_path = root / "labels" / split / f"{img_id}.txt"

        # Download image
        if url:
            safe_download(url, img_path)

        # Write labels
        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)
        W, H = img_info["width"], img_info["height"]

        with open(label_path, "w") as f:
            for ann in anns:
                cat = coco.loadCats([ann["category_id"]])[0]
                cls = map_category_to_3(cat["name"], cat["supercategory"])
                if cls is None:
                    continue

                x, y, w, h = ann["bbox"]
                xc = (x + w/2) / W
                yc = (y + h/2) / H
                nw = w / W
                nh = h / H

                if nw <= 0 or nh <= 0:
                    continue

                f.write(f"{cls} {xc:.6f} {yc:.6f} {nw:.6f} {nh:.6f}\n")
                kept_labels += 1

    print(f"[INFO] Prepared {n_images} images")
    print(f"[INFO] Wrote {kept_labels} bounding boxes into YOLO labels")
    return root


# Bounding box checker

def check_dataset(root="dataset"):
    label_files = glob.glob(f"{root}/labels/**/*.txt", recursive=True)
    total_boxes = 0
    counts = Counter()

    for lf in label_files:
        with open(lf) as f:
            for line in f:
                cls = int(line.split()[0])
                counts[cls] += 1
                total_boxes += 1

    print(f"Number of label files: {len(label_files)}")
    print(f"Total bounding boxes: {total_boxes}")
    print("Boxes per class:", counts)


# Example usage

root = prepare_taco_3class("dataset", n_images=120)
check_dataset(root)


In [9]:
# we have to change the root path here
def train_yolo(root="/content/drive/MyDrive/Colab Notebooks/datasets",
               yaml_name="taco.yaml", epochs=100):
    root = Path(root).resolve()       # absolute dataset folder
    data_yaml = root / yaml_name      # full path to taco.yaml

    # Write YAML with absolute paths
    with open(data_yaml, "w") as f:
        f.write(
            f"train: {root}/images/train\n"
            f"val: {root}/images/val\n"
            "nc: 3\n"
            "names: [metal, plastic, paper]\n"
        )

    print(f"[INFO] Wrote dataset config to {data_yaml}")

    model = yl("yolov8n.pt")
    model.train(data=str(data_yaml), epochs=epochs, imgsz=640)

    print("Training finished. Best model saved in runs/detect/train*/weights/best.pt")
    return model


In [11]:
model = train_yolo(epochs=100)

[INFO] Wrote dataset config to /content/drive/MyDrive/Colab Notebooks/datasets/taco.yaml
Ultralytics 8.3.185 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Colab Notebooks/datasets/taco.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=N

train: Scanning /content/drive/MyDrive/Colab Notebooks/datasets/labels/train.cache... 95 images, 5 backgrounds, 1 corrupt: 100%|██████████| 96/96 [00:00<?, ?it/s]

train: /content/drive/MyDrive/Colab Notebooks/datasets/images/train/297.jpg: corrupt JPEG restored and saved
train: /content/drive/MyDrive/Colab Notebooks/datasets/images/train/315.jpg: ignoring corrupt image/label: image file is truncated (7 bytes not processed)
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 1.1±1.4 ms, read: 318.0±173.8 MB/s, size: 2957.0 KB)


val: Scanning /content/drive/MyDrive/Colab Notebooks/datasets/labels/val.cache... 24 images, 1 backgrounds, 0 corrupt: 100%|██████████| 24/24 [00:00<?, ?it/s]


...

    100/100      2.45G     0.5165     0.6858     0.8647         25        640: 100%|██████████| 6/6 [00:01<00:00,  3.70it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.92it/s]

                   all         24         59       0.19      0.265      0.184      0.121



100 epochs completed in 0.098 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.3MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.185 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.20it/s]


                   all         24         59      0.221      0.145      0.181      0.126
                 metal         16         41      0.664      0.434      0.447      0.316
               plastic          6         11          0          0     0.0187     0.0156
                 paper          6          7          0          0      0.077      0.045
Speed: 0.2ms preprocess, 3.4ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/train2
[INFO] Training finished. Best model saved in runs/detect/train*/weights/best.pt
